# Лабораторная работа №3

Выполнила Вероника Царева, группа БКЛ223.

При работе использовался python версии 3.8.4

### Задание

Разработка системы транслитерации для корейского языка.

**Система транслитерации**

В данной лабораторной работе я использовала [Йельскую романизацию корейского языка](https://en.wikipedia.org/wiki/Yale_romanization_of_Korean#cite_note-FOOTNOTELeeRamsey2000xii-1), так как согласно [Lee, Ramsey 2000]:

> «Yale Romanization, on the other hand, is the romanization of choice for linguists, because it is constructed to reflect linguistic structure.»

Подробная таблица соответствий и комментариев приведена ниже на основе [Sohn 2001: 2-4].

**Комментарии к реализации**

В корейском языке даже для транслитерации важно, с чего начинается слог. Так, он не может начинаться с гласного звука –– перед ним должен предшествовать `ㅇ`, который в начале слога никак не транслитерируется. В других же позициях `ㅇ` транслитерируется как `ng`. 

Для остальных букв есть однозначное соответствие (см. [Sohn 2001: 2-4]).

По этой причине я решила сделать трансдьюсер, который будет транслитерировать один слог. Таким образом, для того чтобы обработать слово целиком, мне необходимо разбить его на слоги и по очереди провести их транслитерацию.

При обработке слов, записанных на хангыле, я заметила, что слог закодирован неделимым символом, поэтому мне не понадобилось руками разбивать каждое слово на слоги при вводе.

Однако это вскрыло другую проблему –– без декомпозиции слога на отдельные буквы я не смогу провести транслитерацию. Для этого был использован модуль [Python-Jamo](https://python-jamo.readthedocs.io/en/latest/#).

**Описание трансдьюсера**

Начальное состояние `q0` –– это первая буква слога. В этом слоте могут быть только согласные. При этом `ㅇ` транслитерируется пустой строкой.

Состояние `q1` –– это любая НЕ первая буква слога. В этом слоте могут быть любые согласные и гласные. При этом `ㅇ` транслитерируется как `ng`. Дополнительно могут встречаться так называемые double bachim –– это сочетание двух букв на конце слога не парсится на части, поэтому транслитерация для них прописана в словаре отдельно.

В конце каждого слога я добавляю символ `§` в качестве флага, что слог закончен. После этого необходимо вернуться в начальное состояние `q0` для обработки следующих слогов. 

**Иллюстрация трансдьюсера**

<img src="https://raw.githubusercontent.com/veronikatsareva/images/main/transducer_illustration.png" alt='transducer for Korean' width='700'/>


**Словарь соответствий**

Ниже приведены словари соответствий, которые используются при переходе из одного состояния в другое (за исключением случая с `ㅇ`, который прописан отдельно на иллюстрации).

```
consonants = {
    "ㅂ": "p",
    "ㅍ": "ph",
    "ㅃ": "pp",
    "ㄷ": "t",
    "ㅌ": "th",
    "ㄸ": "tt",
    "ㅅ": "s",
    "ㅆ": "ss",
    "ㅈ": "c",
    "ㅊ": "ch",
    "ㅉ": "cc",
    "ㄱ": "k",
    "ㅋ": "kh",
    "ㄲ": "kk",
    "ㅁ": "m",
    "ㄴ": "n",
    "ㄹ": "l",
    "ㅎ": "h",
}

vowels = {
    "ㅣ": "i",
    "ㅟ": "wi",
    "ㅔ": "ey",
    "ㅖ": "yey",
    "ㅞ": "wey",
    "ㅚ": "oy",
    "ㅐ": "ay",
    "ㅒ": "yay",
    "ㅙ": "way",
    "ㅡ": "u",
    "ㅓ": "e",
    "ㅕ": "ye",
    "ㅝ": "we",
    "ㅏ": "a",
    "ㅑ": "ya",
    "ㅘ": "wa",
    "ㅜ": "wu",
    "ㅠ": "ywu",
    "ㅗ": "o",
    "ㅛ": "yo",
    "ㅢ": "uy",
}

double_batchim = {
    "ㄳ": "ks",
    "ㄺ": "lk",
    "ㄵ": "nc",
    "ㄶ": "nh",
    "ㄼ": "lp",
    "ㄽ": "ls",
    "ㄾ": "lth",
    "ㅀ": "lh",
    "ㄻ": "lm",
    "ㅄ": "ps",
    "ㄿ": "lph",
}

```

**Источники**

1. Sohn, Ho-Min (2001). The Korean Language. Cambridge University Press.
2. Lee, Iksop; Ramsey, S. Robert (2000). The Korean Language. SUNY Press.

### Реализация

Установка и импорт необходимых модулей.

In [1]:
%pip install jamo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import string
import typing
from jamo import h2j, j2hcj


Создание названий для использующихся типов в функциях.

In [3]:
Transitions = typing.Dict[typing.Tuple[str, str], str]


Словарь переходов.

In [4]:
transitions = {
    # initial
    ("q0", "ㅂ:p"): "q1",
    ("q0", "ㅍ:ph"): "q1",
    ("q0", "ㅃ:pp"): "q1",
    ("q0", "ㄷ:t"): "q1",
    ("q0", "ㅌ:th"): "q1",
    ("q0", "ㄸ:tt"): "q1",
    ("q0", "ㅅ:s"): "q1",
    ("q0", "ㅆ:ss"): "q1",
    ("q0", "ㅈ:c"): "q1",
    ("q0", "ㅊ:ch"): "q1",
    ("q0", "ㅉ:cc"): "q1",
    ("q0", "ㄱ:k"): "q1",
    ("q0", "ㅋ:kh"): "q1",
    ("q0", "ㄲ:kk"): "q1",
    ("q0", "ㅁ:m"): "q1",
    ("q0", "ㄴ:n"): "q1",
    ("q0", "ㅇ:"): "q1",  # ㅇ is not romanized (as syllabic initial)
    ("q0", "ㄹ:l"): "q1",
    ("q0", "ㅎ:h"): "q1",
    # NOT initial
    ("q1", "ㅣ:i"): "q1",
    ("q1", "ㅟ:wi"): "q1",
    ("q1", "ㅔ:ey"): "q1",
    ("q1", "ㅖ:yey"): "q1",
    ("q1", "ㅞ:wey"): "q1",
    ("q1", "ㅚ:oy"): "q1",
    ("q1", "ㅐ:ay"): "q1",
    ("q1", "ㅒ:yay"): "q1",
    ("q1", "ㅙ:way"): "q1",
    ("q1", "ㅡ:u"): "q1",
    ("q1", "ㅓ:e"): "q1",
    ("q1", "ㅕ:ye"): "q1",
    ("q1", "ㅝ:we"): "q1",
    ("q1", "ㅏ:a"): "q1",
    ("q1", "ㅑ:ya"): "q1",
    ("q1", "ㅘ:wa"): "q1",
    ("q1", "ㅜ:wu"): "q1",
    ("q1", "ㅠ:ywu"): "q",
    ("q1", "ㅗ:o"): "q1",
    ("q1", "ㅛ:yo"): "q1",
    ("q1", "ㅢ:uy"): "q1",
    ("q1", "ㅇ:ng"): "q1",  # ㅇ is romanized as ng (as syllabic final)
    ("q1", "ㅂ:p"): "q1",
    ("q1", "ㅍ:ph"): "q1",
    ("q1", "ㅃ:pp"): "q1",
    ("q1", "ㄷ:t"): "q1",
    ("q1", "ㅌ:th"): "q1",
    ("q1", "ㄸ:tt"): "q1",
    ("q1", "ㅅ:s"): "q1",
    ("q1", "ㅆ:ss"): "q1",
    ("q1", "ㅈ:c"): "q1",
    ("q1", "ㅊ:ch"): "q1",
    ("q1", "ㅉ:cc"): "q1",
    ("q1", "ㄱ:k"): "q1",
    ("q1", "ㅋ:kh"): "q1",
    ("q1", "ㄲ:kk"): "q1",
    ("q1", "ㅁ:m"): "q1",
    ("q1", "ㄴ:n"): "q1",
    ("q1", "ㄹ:l"): "q1",
    ("q1", "ㅎ:h"): "q1",
    # double batchim
    ("q1", "ㄳ:ks"): "q1",
    ("q1", "ㄺ:lk"): "q1",
    ("q1", "ㄵ:nc"): "q1",
    ("q1", "ㄶ:nh"): "q1",
    ("q1", "ㄼ:lp"): "q1",
    ("q1", "ㄽ:ls"): "q1",
    ("q1", "ㄾ:lth"): "q1",
    ("q1", "ㅀ:lh"): "q1",
    ("q1", "ㄻ:lm"): "q1",
    ("q1", "ㅄ:ps"): "q1",
    ("q1", "ㄿ:lph"): "q1",
    # end of the syllable
    ("q1", "§:"): "q0",
}


Функция для перехода в новое состояние.

In [5]:
def change_state(
    transitions: Transitions, current_state: str, current_char: str
) -> typing.Tuple[str, str]:
    # перебираем все состояния в словаре переходов
    for (state, encoding), transition in transitions.items():
        # если текущее состояние и текущий символ есть в словаре
        if state == current_state and encoding[0] == current_char:
            # то возращаем транслитерацию текущего символа и переходим
            # в новое состояние
            return transition, encoding[2:]
    # иначе –– ничего
    return None, ""


Функция для обработки предложения на корейском языке.

In [6]:
def transliteration(
    transitions: Transitions,
    current_state: str,
    sentence: str,
    punctuation: typing.List[str],
) -> str:
    # список для хранения транслитерированного предложения
    sentence_transformed = []

    # перебираем все слова в предложении
    for word in sentence.split():
        # список для хранения транслитерированного слова
        current_word = []
        # перебираем все слоги в слове
        for syllable in word:
            # делаем декомпозицию слога на буквы + добавляем в конце
            # слога флаг окончания
            for char in j2hcj(h2j(syllable)) + "§":
                # если текущий знак –– не знак пунктуации
                if char not in punctuation:
                    # то вызываем функцию для транслитерации слога
                    current_state, transformed_char = change_state(
                        transitions, current_state, char
                    )
                    # добавляем обработанную букву в список "слова"
                    current_word.append(transformed_char)

        # добавляем обработанное слово в список "предложения" в виде строки
        sentence_transformed.append("".join(current_word))

    # возвращаем транслитерированное предложение в виде строки
    return " ".join(sentence_transformed)


### Примеры использования

In [7]:
# список слов/фраз на корейском языке
# в комментариях приведен перевод и то, как слова должны быть по итогу
# транслитерированы

tests = [
    "",  # проверка корректной обработки пустой строки
    "한국",  # Hankwuk 'Korea'
    "한굴",  # Hankul 'Korean alphabet'
    "서울",  # Sewul 'Seoul'
    "안녕",  # annyeng 'hi'
    "안녕하세요 감사합니다",  # annyenghaseyyo kamsahapnita 'Hello, thank you'
    "바다",  # pata 'sea, ocean'
    "산",  # san 'mountain'
    "가자",  # kaca 'let's go'
    "에이티즈",  # eyithicu 'ATEEZ'
    "괜찮아",  # kwaynchanha 'it's alright'
]


In [8]:
# множество знаков пунктуации
punctuation = set(string.punctuation)

# начальное состояние
start_state = "q0"

# вызов функции для транслитерации каждого примера из списка
for phrase in tests:
    print(
        f"{phrase} --> {transliteration(transitions, start_state, phrase, punctuation)}"
    )


 --> 
한국 --> hankwuk
한굴 --> hankwul
서울 --> sewul
안녕 --> annyeng
안녕하세요 감사합니다 --> annyenghaseyyo kamsahapnita
바다 --> pata
산 --> san
가자 --> kaca
에이티즈 --> eyithicu
괜찮아 --> kwaynchanha
